In [5]:
from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad
import os
import time
from PIL import Image
import struct
import hashlib  # For data integrity check

def load_image(image_path):
    """Loads image data, size, and mode."""
    try:
        img = Image.open(image_path)
        return img.tobytes(), img.size, img.mode
    except FileNotFoundError:
        raise FileNotFoundError(f"Error: Image not found at {image_path}")
    except Exception as e:
        raise Exception(f"Error loading image: {e}")

def save_image(image_data, size, mode, output_path):
    """Saves byte data as an image."""
    try:
        Image.frombytes(mode, size, image_data).save(output_path)
        print(f"Image saved to {output_path}")
    except Exception as e:
        raise Exception(f"Error saving image: {e}")

def encrypt_image_des_pycrypto(image_path, key):
    """Encrypts an image using DES in CBC mode with pycryptodome."""
    image_bytes, size, mode = load_image(image_path)
    if len(key) != 8:
        raise ValueError("Key must be 8 bytes long for DES")

    iv = os.urandom(DES.block_size)
    cipher = DES.new(key, DES.MODE_CBC, iv)
    padded_data = pad(image_bytes, DES.block_size)
    start_time = time.time()
    ciphertext = iv + cipher.encrypt(padded_data)
    end_time = time.time()
    encryption_time = end_time - start_time

    size_bytes = struct.pack("<II", size[0], size[1])
    mode_bytes = mode.encode('utf-8')
    mode_length = len(mode_bytes)
    if mode_length > 255:
        raise ValueError("Image mode string is too long (max 255 bytes)")
    encrypted_data = size_bytes + bytes([mode_length]) + mode_bytes + ciphertext
    return encrypted_data, size, mode, encryption_time

def decrypt_image_des_pycrypto(encrypted_data, key):
    """Decrypts an image encrypted with DES in CBC mode with pycryptodome."""
    if len(key) != 8:
        raise ValueError("Key must be 8 bytes long for DES")
    if len(encrypted_data) < DES.block_size + 8 + 1:
        raise ValueError("Invalid encrypted data format")

    size_bytes = encrypted_data[:8]
    size = struct.unpack("<II", size_bytes)
    mode_length = encrypted_data[8]
    mode_bytes = encrypted_data[9 : 9 + mode_length]
    mode = mode_bytes.decode('utf-8')
    ciphertext = encrypted_data[9 + mode_length :]

    if len(ciphertext) < DES.block_size:
        raise ValueError("Invalid encrypted data format. Ciphertext too short")

    iv = ciphertext[:DES.block_size]
    ciphertext_body = ciphertext[DES.block_size:]
    cipher = DES.new(key, DES.MODE_CBC, iv)
    start_time = time.time()
    decrypted_padded_data = cipher.decrypt(ciphertext_body)
    decrypted_data = unpad(decrypted_padded_data, DES.block_size)
    end_time = time.time()
    decryption_time = end_time - start_time
    return decrypted_data, size, mode, decryption_time

def calculate_md5_hash(data):
    """Calculates the MD5 hash of the given data."""
    md5_hash = hashlib.md5(data).hexdigest()
    return md5_hash

def compare_images(original_image_path, decrypted_image_path):
    """Compares the original and decrypted images and returns True if they are identical, False otherwise."""
    try:
        original_img = Image.open(original_image_path)
        decrypted_img = Image.open(decrypted_image_path)
        if original_img.size != decrypted_img.size or original_img.mode != decrypted_img.mode:
            return False, "Sizes or modes differ"
        original_data = original_img.tobytes()
        decrypted_data = decrypted_img.tobytes()
        return original_data == decrypted_data, "Images are identical" if original_data == decrypted_data else "Images are different"
    except Exception as e:
        return False, f"Error comparing images: {e}"

if __name__ == "__main__":
    image_file = input("Enter the path to the image file: ")
    key_str = input("Enter the 8-byte encryption key: ")
    if len(key_str.encode('utf-8')) != 8:
        print("Error: Key must be 8 bytes long for DES.")
    else:
        key = key_str.encode('utf-8')
        operation = input("Enter 'encrypt' or 'decrypt': ").lower()

        if operation == "encrypt":
            try:
                encrypted_data, original_size, original_mode, encryption_time = encrypt_image_des_pycrypto(image_file, key)
                original_image_bytes, _, _ = load_image(image_file)  # Load before saving encrypted
                original_image_md5 = calculate_md5_hash(original_image_bytes)
                output_file = input("Enter the path to save the encrypted file: ")
                with open(output_file, "wb") as f:
                    f.write(encrypted_data)
                print(f"Image encrypted and saved to {output_file}")
                print(f"Encryption time: {encryption_time:.4f} seconds")  # Display time

            except Exception as e:
                print(f"Error during encryption: {e}")

        elif operation == "decrypt":
            encrypted_file = input("Enter the path to the encrypted file: ")
            try:
                with open(encrypted_file, "rb") as f:
                    encrypted_data = f.read()
                decrypted_data, size, mode, decryption_time = decrypt_image_des_pycrypto(encrypted_data, key)
                output_file = input("Enter the path to save the decrypted image: ")
                save_image(decrypted_data, size, mode, output_file)
                print(f"Decryption time: {decryption_time:.4f} seconds")

                # Data integrity check (MD5 hash)
                decrypted_image_md5 = calculate_md5_hash(decrypted_data)

                # Compare original and decrypted images
                comparison_result, comparison_message = compare_images(image_file, output_file)  # Use image_file
                print(comparison_message)
                print(f"Original image MD5 hash: {original_image_md5}")
                print(f"Decrypted image MD5 hash: {decrypted_image_md5}")
                if comparison_result:
                    print("Data integrity check: PASSED (MD5 hashes match)")
                else:
                    print("Data integrity check: FAILED (MD5 hashes differ)")

            except FileNotFoundError:
                print(f"Error: Encrypted file not found at {encrypted_file}")
            except Exception as e:
                print(f"Error during decryption: {e}")
        else:
            print("Invalid operation. Please enter 'encrypt' or 'decrypt'.")

Enter the path to the image file:  C:\Users\Pratishtha Saha\image.png
Enter the 8-byte encryption key:  mysecret
Enter 'encrypt' or 'decrypt':  encrypt
Enter the path to save the encrypted file:  D:\pictures\encrypted_image.des


Image encrypted and saved to D:\pictures\encrypted_image.des
Encryption time: 0.0025 seconds
